In [ ]:
# 必要なモジュールをimportする
import requests
import re
import urllib
import ast
import json
import pprint

In [ ]:
instances_str = '''ここにURLを入力(ドメインの終わりには/が必須 e.g. https://google.com/search?q=exsample)'''

In [ ]:
# e.g. "https://www.google.com/?q=exsamplehttps://www.apple.com/"
#      => ["https://www.google.com/", "https://www.apple.com/"]
# Note: httpの場合httpsに変換して処理する
#       必ず有効なURLとは限らない（あくまで抽出するだけ）

def extractionUrlsFromStr(str):
  splitted_urls = str.replace('http://', 'https://').split('https://')

  unpermitted_characters_in_url = r'([^\w!\?\/\+\\\-~;\.,\*&@#\$%\(\)\'\[\]])'
  unpermitted_characters_re = re.compile(unpermitted_characters_in_url)

  urls = []

  for unformatted_str in splitted_urls:
    formatted_url = re.sub(unpermitted_characters_re, '', unformatted_str)

    urls.append('https:// ' + urllib.parse.urlparse(formatted_url).netloc + '/')

  return urls

In [ ]:
instance_urls = extractionUrlsFromStr(instances_str)

pprint.pprint(f'Invidiousのサイト数: {len(instances_urls)}\\nURL: {instances_urls}')

In [ ]:
success_instances = []

for url in instance_urls:
  try:
    res = requests.get(url + 'api/v1/videos/shs0rAiwsGQ', timeout=(3.0, 1.5)).text
    json_res = json.loads(res)
    if 'error' not in json_res:
      video_res = requests.get(json_res['formatStreams'][0]['url'], timeout=(1.0, 0.5))
      if video_res.headers['Content-Type'] == 'video/mp4':
        print('SUCCESS: ' + url)
        success_instances.append(url)
      else:
        print('not video: ' + url)
    else:
      print('instances returned error: ' + url)

  except:
    print('ERROR: ' + url)
    pass


In [ ]:
pprint.pprint(success_instances)